In [ ]:

import os
from google.colab import drive
drive.mount('/gdrive')




Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!fusermount -u drive
!google-drive-ocamlfuse drive

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from pandas.core.common import flatten
import copy
import numpy as np
import random

import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

from albumentations.pytorch import ToTensorV2
import cv2

import glob
from tqdm import tqdm

fusermount: failed to unmount /content/drive: No such file or directory
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
# Model
from torchvision.models import resnet50, ResNet50_Weights

import torch.nn as nn
import torchvision.models as models

class inceptionv3(nn.Module):
    def __init__(self, train_inceptionv3=False, num_classes=3):
        super(inceptionv3, self).__init__()
        self.train_inceptionv3 = train_inceptionv3
        self.inception = models.inception_v3(pretrained=True, aux_logits=True)
        self.inception.dropout=nn.Dropout(0.3)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, 1024)
        self.dropout = nn.Dropout(0.2)
        self.linear =nn.Linear(1024,num_classes)

    def forward(self, images):
        features = self.dropout(self.inception(images))
        features_2 = self.linear(features)
        return features_2.squeeze(1)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from tqdm import tqdm
device = ("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Transform
transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

In [ ]:
# !pip install kora -q
# from kora import drive
# drive.link_nbs()





     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 1.6 MB 26.3 MB/s 
Mounted at /content/drive


In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Dataset class
from torch.utils.data import Dataset
import pandas as pd
import numpy
import os
from PIL import Image
import torch

class Damage_location(Dataset):
    def __init__(self, root_dir_front,root_dir_side, root_dir_rear, annotation_file, transform=None):
        self.root_dir1 = root_dir_front
        self.root_dir2 = root_dir_rear
        self.root_dir3 = root_dir_side
        self.annotations = pd.read_csv('/gdrive/MyDrive/project/'+annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        # print(self.annotations.iloc[index, 1][1:-1])
      

        value = numpy.fromstring(self.annotations.iloc[index, 1][1:-1], dtype=float, sep=' ')
        # print(value)

        value = torch.from_numpy(value)
        # print(type(value))

        # print(value)
        if value[0] == 1:
          root_dir = self.root_dir1
        elif value[2] == 1:
          root_dir = self.root_dir2
        else:
          root_dir = self.root_dir3
        img = Image.open(os.path.join(root_dir, img_id)).convert("RGB")
        npp = numpy.fromstring(self.annotations.iloc[index, 1] [1:-1],dtype=float,sep = ' ')
        y_label = torch.from_numpy((npp))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [ ]:
root2= '/gdrive/MyDrive/project/car-damage-dataset/data2a'
training = root2+'/training'
# we then split the training in train, valid so we store everything in the training folder
front = '/00-front/'
rear = '/01-rear/'
side = '/02-side/'


In [ ]:
num_epochs = 40
learning_rate = 0.02
train_CNN = False
batch_size = 64
shuffle = True
pin_memory = True
num_workers = 1


In [ ]:
dataset = Damage_location(training+front,training+side,training+rear,"damage_locations_3.csv",transform=transform)
train_set, validation_set = torch.utils.data.random_split(dataset,[960,190])
train_loader = DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)

In [ ]:
print(train_set)

In [ ]:
model = inceptionv3().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for name, param in model.inception.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = train_CNN

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# only save new weights if the accuracy is > 75% 
max_accuracy = 0.75

In [ ]:
print(model.parameters)

In [ ]:
import numpy

In [ ]:
# Function used for evaluating during training
def check_accuracy(loader, model):
    if loader == train_loader:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on validation data")
    num_classes =3
    num_correct = 0
    num_samples = 0
    model.eval()
    root = '/gdrive/MyDrive/project/car_damage_location/'
    with torch.no_grad():
        for x, y in loader:
            # print('hhh')
            x = x.to(device=device)
            y = y.to(device=device)

            # x =x.type(torch.FloatTensor)
            # y =y.type(torch.FloatTensor)

            scores = model(x)


            # predictions = torch.tensor([1.0 if i >= 0.5 else 0.0 for i in scores]).to(device)
            t1 = torch.tensor([1.,0,0])
            t2 = torch.tensor([0,1.,0])
            t3 = torch.tensor([0,0,1.])
            prediction_list =[]
            for score in scores:
              max_index = score.argmax()
              if max_index == 0:
                prediction_list.append(t1)
              elif max_index ==1:
                prediction_list.append(t2)
              else:
                prediction_list.append(t3)
            # print(prediction_list)
            predictions = torch.stack(prediction_list).to(device)
            
            for sample in range(predictions.size()[0]):
              if torch.equal(predictions[sample],y[sample]):
                num_correct += 1
            num_samples += predictions.size(0)
            print('num of correct: ', num_correct, ' num of samples: ', num_samples)
            ratio = float(num_correct)/float(num_samples)
            if loader != train_loader:
              global max_accuracy
              if ratio>= max_accuracy:
                max_accuracy = ratio
                print('max accuracy is:', max_accuracy)
                torch.save(model.state_dict(), root+'model_location_weights_final_2.pth')
    return f"{float(num_correct)/float(num_samples)*100:.2f}"
    print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
    model.train()

In [ ]:
# Training function
def train():
    model.train()
    for epoch in range(num_epochs):
        loop = tqdm(train_loader, total = len(train_loader), leave = True)
        if epoch % 2 == 0:
            loop.set_postfix(val_acc = check_accuracy(validation_loader, model))
        for imgs, labels in loop:
            imgs = imgs.to(device)
            labels = labels.to(device)
            labels=labels.to(torch.int64)
            outputs = model(imgs)
            loss = criterion(outputs, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
            loop.set_postfix(loss = loss.item())


In [ ]:
if __name__ == "__main__":
    train()

In [ ]:
train()

In [ ]:
print(max_accuracy)

0.75
